## 1. Setup & Dependencies

### 1.1. Install Required Packages

In [1]:
# !pip install matplotlib
# !pip install pyserini
# !pip install faiss-cpu
# !pip install torch
# !pip install transformers

In [2]:
import os
import warnings
from abc import ABC, abstractmethod
from typing import Dict, List, Tuple, Optional
from collections import defaultdict
from itertools import product

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tqdm import tqdm
tqdm.pandas()

from pyserini.search.lucene import LuceneSearcher
from pyserini.index.lucene import IndexReader

import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

warnings.filterwarnings("ignore")

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {DEVICE}")

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

print("✓ Dependencies imported")

/home/galnoy/git-projects/MSC-Text-Retrieval-and-Search-Engines/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2026-01-09 15:43:23.520028349 [W:onnxruntime:Default, device_discovery.cc:164 DiscoverDevicesForPlatform] GPU device discovery failed: device_discovery.cc:89 ReadFileContents Failed to open file: "/sys/class/drm/card0/device/vendor"


Using device: cuda
✓ Dependencies imported


### 1.2. Load Pyserini Index

In [3]:
INDEX_NAME = "robust04"

index_reader = IndexReader.from_prebuilt_index(INDEX_NAME)

print(f"Index: {INDEX_NAME}")
print(f"Total documents: {index_reader.stats()['documents']:,}")
print(f"Total terms: {index_reader.stats()['total_terms']:,}")
print("✓ Pyserini index loaded")

Jan 09, 2026 3:43:24 PM org.apache.lucene.store.MemorySegmentIndexInputProvider <init>
INFO: Using MemorySegmentIndexInput with Java 21; to disable start with -Dorg.apache.lucene.store.MMapDirectory.enableMemorySegments=false


Index: robust04
Total documents: 528,030
Total terms: 174,540,872
✓ Pyserini index loaded


## 2. Data Loading

### 2.1. Load Queries

In [4]:
DATA_DIR = "./data/"

def load_queries(filepath: str) -> Dict[str, str]:
    """Load queries from file. Format: qid<tab>query_text"""
    queries = {}
    with open(filepath, "r") as f:
        for line in f:
            parts = line.strip().split("\t")
            if len(parts) == 2:
                qid, text = parts
                queries[qid] = text
    return queries

all_queries = load_queries(os.path.join(DATA_DIR, "queriesROBUST.txt"))

print(f"Total queries loaded: {len(all_queries)}")
print(f"\nSample queries:")
for qid, text in list(all_queries.items())[:5]:
    print(f"  {qid}: {text}")

Total queries loaded: 249

Sample queries:
  301: international organized crime
  302: poliomyelitis post polio
  303: hubble telescope achievements
  304: endangered species mammals
  305: dangerous vehicles


### 2.2. Load Relevance Judgments

In [5]:
def load_qrels(filepath: str) -> Dict[str, Dict[str, int]]:
    """Load qrels. Format: qid 0 docid relevance"""
    qrels = defaultdict(dict)
    with open(filepath, "r") as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) >= 4:
                qid, _, docid, rel = parts[:4]
                qrels[qid][docid] = int(rel)
    return dict(qrels)

qrels = load_qrels(os.path.join(DATA_DIR, "qrels_50_Queries"))

print(f"Queries with relevance judgments: {len(qrels)}")
print(f"Total judgments: {sum(len(v) for v in qrels.values()):,}")

Queries with relevance judgments: 50
Total judgments: 61,511


### 2.3. Train/Test Split

In [6]:
train_qids = sorted(qrels.keys())
test_qids = [qid for qid in all_queries.keys() if qid not in train_qids]

train_queries = {qid: all_queries[qid] for qid in train_qids}
test_queries = {qid: all_queries[qid] for qid in test_qids}

print(f"Training queries: {len(train_queries)} (with qrels)")
print(f"Test queries: {len(test_queries)} (no qrels)")

Training queries: 50 (with qrels)
Test queries: 199 (no qrels)


## 3. Evaluation Metrics

In [7]:
def compute_ap(ranked_docs: List[str], relevance: Dict[str, int]) -> float:
    """Compute Average Precision for a single query."""
    relevant = {d for d, r in relevance.items() if r > 0}
    if not relevant:
        return 0.0
    
    hits = 0
    precision_sum = 0.0
    
    for i, doc in enumerate(ranked_docs):
        if doc in relevant:
            hits += 1
            precision_sum += hits / (i + 1)
    
    return precision_sum / len(relevant)


def compute_map(
    run: Dict[str, List[Tuple[str, float]]],
    qrels: Dict[str, Dict[str, int]]
) -> float:
    """Compute Mean Average Precision over all queries."""
    aps = []
    for qid, results in run.items():
        if qid in qrels:
            ranked_docs = [doc for doc, _ in results]
            ap = compute_ap(ranked_docs, qrels[qid])
            aps.append(ap)
    return np.mean(aps) if aps else 0.0


def evaluate_run(
    run: Dict[str, List[Tuple[str, float]]],
    qrels: Dict[str, Dict[str, int]]
) -> Dict[str, float]:
    """Evaluate a run and return metrics."""
    map_score = compute_map(run, qrels)
    
    per_query = {}
    for qid in run:
        if qid in qrels:
            ranked = [d for d, _ in run[qid]]
            per_query[qid] = compute_ap(ranked, qrels[qid])
    
    return {
        "map": map_score,
        "num_queries": len(per_query),
        "per_query_ap": per_query
    }


print("✓ Evaluation metrics defined")

✓ Evaluation metrics defined


## 4. Retrieval Models

### 4.1. Abstract Base Class

In [8]:
class BaseRetriever(ABC):
    """
    Abstract base class for all retrieval models.
    All models must implement search.
    """
    
    def __init__(self, index_name: str = "robust04"):
        self.index_name = index_name
    
    @abstractmethod
    def search(
        self,
        queries: Dict[str, str],
        k: int = 1000
    ) -> Dict[str, List[Tuple[str, float]]]:
        """Search for all queries and return ranked results."""
        pass
    
    def get_params(self) -> Dict:
        """Return model parameters for logging."""
        return {}


print("✓ BaseRetriever class defined")

✓ BaseRetriever class defined


### 4.2. BM25 Retriever

In [9]:
class BM25Retriever(BaseRetriever):
    """
    BM25 retrieval model.
    
    BM25 score = sum over terms t in q:
        IDF(t) * (tf(t,d) * (k1 + 1)) / (tf(t,d) + k1 * (1 - b + b * |d|/avgdl))
    """
    
    def __init__(
        self,
        k1: float = 0.9,
        b: float = 0.4,
        index_name: str = "robust04"
    ):
        super().__init__(index_name)
        self.k1 = k1
        self.b = b
        self._searcher = None
    
    @property
    def searcher(self) -> LuceneSearcher:
        if self._searcher is None:
            self._searcher = LuceneSearcher.from_prebuilt_index(self.index_name)
            self._searcher.set_bm25(k1=self.k1, b=self.b)
        return self._searcher
    
    def search(
        self,
        queries: Dict[str, str],
        k: int = 1000
    ) -> Dict[str, List[Tuple[str, float]]]:
        results = {}
        for qid, query_text in tqdm(queries.items(), desc="BM25 Search"):
            hits = self.searcher.search(query_text, k=k)
            results[qid] = [(hit.docid, hit.score) for hit in hits]
        return results
    
    def get_params(self) -> Dict:
        return {"k1": self.k1, "b": self.b}


print("✓ BM25Retriever class defined")

✓ BM25Retriever class defined


### 4.3. BM25 + RM3 Retriever (Query Expansion)

In [10]:
class RM3Retriever(BaseRetriever):
    """
    BM25 + RM3 pseudo-relevance feedback.
    
    RM3 expands the query using terms from top-k retrieved documents.
    Final query = original_weight * original_query + (1 - original_weight) * expansion_terms
    """
    
    def __init__(
        self,
        k1: float = 0.9,
        b: float = 0.4,
        fb_terms: int = 10,
        fb_docs: int = 10,
        original_weight: float = 0.5,
        index_name: str = "robust04"
    ):
        super().__init__(index_name)
        self.k1 = k1
        self.b = b
        self.fb_terms = fb_terms
        self.fb_docs = fb_docs
        self.original_weight = original_weight
        self._searcher = None
    
    @property
    def searcher(self) -> LuceneSearcher:
        if self._searcher is None:
            self._searcher = LuceneSearcher.from_prebuilt_index(self.index_name)
            self._searcher.set_bm25(k1=self.k1, b=self.b)
            self._searcher.set_rm3(
                fb_terms=self.fb_terms,
                fb_docs=self.fb_docs,
                original_query_weight=self.original_weight
            )
        return self._searcher
    
    def search(
        self,
        queries: Dict[str, str],
        k: int = 1000
    ) -> Dict[str, List[Tuple[str, float]]]:
        results = {}
        for qid, query_text in tqdm(queries.items(), desc="RM3 Search"):
            hits = self.searcher.search(query_text, k=k)
            results[qid] = [(hit.docid, hit.score) for hit in hits]
        return results
    
    def get_params(self) -> Dict:
        return {
            "k1": self.k1,
            "b": self.b,
            "fb_terms": self.fb_terms,
            "fb_docs": self.fb_docs,
            "original_weight": self.original_weight
        }


print("✓ RM3Retriever class defined")

✓ RM3Retriever class defined


### 4.4. Hybrid Neural Retriever (Advanced)

This method combines BM25 first-stage retrieval with neural cross-encoder re-ranking, then fuses the results using Reciprocal Rank Fusion (RRF):

$$
\text{RRF}(d) = \sum_{r \in R} \frac{1}{k + \text{rank}_r(d)}
$$

where $R$ is the set of ranking systems and $k$ is a constant (typically 60).

In [11]:
class HybridNeuralRetriever(BaseRetriever):
    """
    Hybrid retrieval: BM25 -> Neural Re-ranking -> RRF Fusion.
    
    Uses a cross-encoder model to re-rank BM25 candidates,
    then fuses BM25 and neural rankings using RRF.
    """
    
    def __init__(
        self,
        k1: float = 0.9,
        b: float = 0.4,
        rerank_depth: int = 100,
        rrf_k: int = 60,
        model_name: str = "cross-encoder/ms-marco-MiniLM-L-6-v2",
        index_name: str = "robust04",
        batch_size: int = 32
    ):
        super().__init__(index_name)
        self.k1 = k1
        self.b = b
        self.rerank_depth = rerank_depth
        self.rrf_k = rrf_k
        self.model_name = model_name
        self.batch_size = batch_size
        
        self._searcher = None
        self._tokenizer = None
        self._cross_encoder = None
    
    @property
    def searcher(self) -> LuceneSearcher:
        if self._searcher is None:
            self._searcher = LuceneSearcher.from_prebuilt_index(self.index_name)
            self._searcher.set_bm25(k1=self.k1, b=self.b)
        return self._searcher
    
    @property
    def tokenizer(self):
        if self._tokenizer is None:
            self._tokenizer = AutoTokenizer.from_pretrained(self.model_name)
        return self._tokenizer
    
    @property
    def cross_encoder(self):
        if self._cross_encoder is None:
            self._cross_encoder = AutoModelForSequenceClassification.from_pretrained(self.model_name)
            self._cross_encoder = self._cross_encoder.to(DEVICE)
            self._cross_encoder.eval()
        return self._cross_encoder
    
    def _get_doc_text(self, docid: str) -> str:
        """Retrieve document text from index."""
        doc = self.searcher.doc(docid)
        if doc is None:
            return ""
        raw = doc.raw()
        return raw[:2000] if raw else ""
    
    @torch.no_grad()
    def _rerank(self, query: str, candidates: List[Tuple[str, float]]) -> List[Tuple[str, float]]:
        """Re-rank candidates using cross-encoder."""
        if not candidates:
            return []
        
        docids = [d for d, _ in candidates]
        docs = [self._get_doc_text(d) for d in docids]
        
        scores = []
        for i in range(0, len(docs), self.batch_size):
            batch_docs = docs[i:i+self.batch_size]
            pairs = [[query, doc] for doc in batch_docs]
            
            inputs = self.tokenizer(
                pairs,
                padding=True,
                truncation=True,
                max_length=512,
                return_tensors="pt"
            ).to(DEVICE)
            
            outputs = self.cross_encoder(**inputs)
            batch_scores = outputs.logits.squeeze(-1).cpu().numpy()
            scores.extend(batch_scores.tolist() if batch_scores.ndim > 0 else [batch_scores.item()])
        
        reranked = sorted(zip(docids, scores), key=lambda x: x[1], reverse=True)
        return reranked
    
    def _rrf_fusion(self, runs: List[List[Tuple[str, float]]]) -> List[Tuple[str, float]]:
        """Combine rankings using Reciprocal Rank Fusion."""
        rrf_scores = defaultdict(float)
        
        for run in runs:
            for rank, (docid, _) in enumerate(run):
                rrf_scores[docid] += 1.0 / (self.rrf_k + rank + 1)
        
        fused = sorted(rrf_scores.items(), key=lambda x: x[1], reverse=True)
        return fused
    
    def search(
        self,
        queries: Dict[str, str],
        k: int = 1000
    ) -> Dict[str, List[Tuple[str, float]]]:
        results = {}
        
        for qid, query_text in tqdm(queries.items(), desc="Hybrid Search"):
            # First-stage: BM25
            bm25_hits = self.searcher.search(query_text, k=self.rerank_depth)
            bm25_candidates = [(hit.docid, hit.score) for hit in bm25_hits]
            
            # Second-stage: Neural re-ranking
            neural_reranked = self._rerank(query_text, bm25_candidates)
            
            # Fusion: RRF
            fused = self._rrf_fusion([bm25_candidates, neural_reranked])
            
            # Pad with BM25 results if needed
            if len(fused) < k:
                extra_hits = self.searcher.search(query_text, k=k)
                seen = {d for d, _ in fused}
                for hit in extra_hits:
                    if hit.docid not in seen:
                        fused.append((hit.docid, 0.0))
                        if len(fused) >= k:
                            break
            
            results[qid] = fused[:k]
        
        return results
    
    def get_params(self) -> Dict:
        return {
            "k1": self.k1,
            "b": self.b,
            "rerank_depth": self.rerank_depth,
            "rrf_k": self.rrf_k,
            "model_name": self.model_name
        }


print("✓ HybridNeuralRetriever class defined")

✓ HybridNeuralRetriever class defined


## 5. Hyperparameter Tuning

### 5.1. Experiment Caching Utils

In [16]:
RESULTS_CSV = "./experiments.csv"


def generate_config_key(model_name: str, params: Dict) -> str:
    """Generate a unique config key per model + params."""
    parts = [model_name]
    for k, v in sorted(params.items()):
        if k not in ["index_name", "model_name", "batch_size"]:
            parts.append(f"{k}={v}")
    return "__".join(parts)


def load_completed_experiments() -> pd.DataFrame:
    if not os.path.exists(RESULTS_CSV):
        return pd.DataFrame()
    return pd.read_csv(RESULTS_CSV)


def load_cached_result(config_key: str) -> Optional[Dict]:
    df = load_completed_experiments()
    if df.empty:
        return None
    row = df[df["config_key"] == config_key]
    if row.empty:
        return None
    return row.iloc[0].to_dict()


def save_experiment_result(result: Dict):
    df_row = pd.DataFrame([result])
    if not os.path.exists(RESULTS_CSV):
        df_row.to_csv(RESULTS_CSV, index=False)
    else:
        df_row.to_csv(RESULTS_CSV, mode="a", header=False, index=False)


print("✓ Caching utils ready")
print(f"  Results CSV: {RESULTS_CSV}")

✓ Caching utils ready
  Results CSV: ./experiments.csv


### 5.2. Experiment Framework

In [17]:
def run_experiment(
    config_key: str,
    model_name: str,
    model_class: type,
    model_params: Dict,
    queries: Dict[str, str],
    qrels: Dict[str, Dict[str, int]]
) -> Dict:
    """Run a single experiment and return results."""
    model = model_class(**model_params)
    run = model.search(queries, k=1000)
    metrics = evaluate_run(run, qrels)
    
    return {
        "config_key": config_key,
        "model": model_name,
        "map": metrics["map"],
        "num_queries": metrics["num_queries"]
    }


print("✓ Experiment framework ready")

✓ Experiment framework ready


### 5.3. Hyperparameter Grid Search

In [18]:
EXPERIMENTS = [
    {
        "model_name": "BM25",
        "model_class": BM25Retriever,
        "param_grid": [
            {"k1": k1, "b": b}
            for k1, b in product(
                [0.6, 0.9, 1.2, 1.5, 2.0],
                [0.3, 0.4, 0.5, 0.6, 0.75]
            )
        ],
    },
    {
        "model_name": "RM3",
        "model_class": RM3Retriever,
        "param_grid": [
            {"k1": 0.9, "b": 0.4, "fb_terms": fb_terms, "fb_docs": fb_docs, "original_weight": orig_w}
            for fb_terms, fb_docs, orig_w in product(
                [10, 20, 30],
                [5, 10, 15],
                [0.3, 0.5, 0.7]
            )
        ],
    },
    {
        "model_name": "HybridNeural",
        "model_class": HybridNeuralRetriever,
        "param_grid": [
            {"k1": 0.9, "b": 0.4, "rerank_depth": rerank_depth, "rrf_k": rrf_k}
            for rerank_depth, rrf_k in product(
                [50, 100],
                [30, 60, 90]
            )
        ],
    },
]

print(f"Total experiments configured:")
for exp in EXPERIMENTS:
    print(f"  {exp['model_name']}: {len(exp['param_grid'])} configurations")

Total experiments configured:
  BM25: 25 configurations
  RM3: 27 configurations
  HybridNeural: 6 configurations


In [20]:
results = []

for exp in EXPERIMENTS:
    model_name = exp["model_name"]
    model_class = exp["model_class"]
    
    for params in exp["param_grid"]:
        config_key = generate_config_key(model_name, params)
        
        print("\n" + "=" * 60)
        print(f"Experiment: {config_key}")
        print(f"Model: {model_name}")
        print("=" * 60)
        
        cached = load_cached_result(config_key)
        if cached is not None:
            print("✓ Loaded from cache")
            result = cached
        else:
            result = run_experiment(
                config_key=config_key,
                model_name=model_name,
                model_class=model_class,
                model_params=params,
                queries=train_queries,
                qrels=qrels
            )
            save_experiment_result(result)
        
        print(f"\nResults:")
        print(f"  MAP: {result['map']:.4f}")
        
        results.append(result)

print("\n" + "=" * 60)
print("All experiments completed.")
print("=" * 60)


Experiment: BM25__b=0.3__k1=0.6
Model: BM25
✓ Loaded from cache

Results:
  MAP: 0.2463

Experiment: BM25__b=0.4__k1=0.6
Model: BM25
✓ Loaded from cache

Results:
  MAP: 0.2475

Experiment: BM25__b=0.5__k1=0.6
Model: BM25
✓ Loaded from cache

Results:
  MAP: 0.2453

Experiment: BM25__b=0.6__k1=0.6
Model: BM25
✓ Loaded from cache

Results:
  MAP: 0.2422

Experiment: BM25__b=0.75__k1=0.6
Model: BM25
✓ Loaded from cache

Results:
  MAP: 0.2392

Experiment: BM25__b=0.3__k1=0.9
Model: BM25
✓ Loaded from cache

Results:
  MAP: 0.2450

Experiment: BM25__b=0.4__k1=0.9
Model: BM25
✓ Loaded from cache

Results:
  MAP: 0.2455

Experiment: BM25__b=0.5__k1=0.9
Model: BM25
✓ Loaded from cache

Results:
  MAP: 0.2442

Experiment: BM25__b=0.6__k1=0.9
Model: BM25
✓ Loaded from cache

Results:
  MAP: 0.2415

Experiment: BM25__b=0.75__k1=0.9
Model: BM25
✓ Loaded from cache

Results:
  MAP: 0.2374

Experiment: BM25__b=0.3__k1=1.2
Model: BM25
✓ Loaded from cache

Results:
  MAP: 0.2426

Experiment: BM25__

RM3 Search:   0%|          | 0/50 [00:00<?, ?it/s]

RM3 Search: 100%|██████████| 50/50 [00:02<00:00, 19.41it/s]



Results:
  MAP: 0.2616

Experiment: RM3__b=0.4__fb_docs=5__fb_terms=10__k1=0.9__original_weight=0.7
Model: RM3


RM3 Search: 100%|██████████| 50/50 [00:02<00:00, 19.81it/s]


...


Results:
  MAP: 0.2475

Experiment: HybridNeural__b=0.4__k1=0.9__rerank_depth=50__rrf_k=60
Model: HybridNeural


Hybrid Search: 100%|██████████| 50/50 [00:20<00:00,  2.43it/s]



Results:
  MAP: 0.2465

Experiment: HybridNeural__b=0.4__k1=0.9__rerank_depth=50__rrf_k=90
Model: HybridNeural


Hybrid Search:  44%|████▍     | 22/50 [00:15<00:19,  1.45it/s]


KeyboardInterrupt: 

In [ ]:
results_sorted = sorted(results, key=lambda x: x["map"], reverse=True)
top_k = results_sorted[:10]

print("\n" + "=" * 100)
print("Top 10 Results (sorted by MAP):")
print("=" * 100)

header = f"{'#':<3} {'Model':<15} {'MAP':>10} {'Config':<70}"
print(header)
print("-" * len(header))

for i, r in enumerate(top_k, start=1):
    print(
        f"{i:<3} "
        f"{r['model']:<15} "
        f"{r['map']:>10.4f} "
        f"{r['config_key']:<70}"
    )

## 6. Results Summary

### 6.1. Best Model per Method

In [ ]:
# Find best config for each model type
best_per_model = {}
for r in results_sorted:
    model = r["model"]
    if model not in best_per_model:
        best_per_model[model] = r

print("Best configuration per model:")
print("=" * 80)
for model, r in best_per_model.items():
    print(f"\n{model}:")
    print(f"  MAP: {r['map']:.4f}")
    print(f"  Config: {r['config_key']}")

### 6.2. Training Performance Comparison

In [ ]:
summary_data = []
for model, r in best_per_model.items():
    summary_data.append({
        "Method": model,
        "MAP": r["map"],
        "Config": r["config_key"]
    })

summary_df = pd.DataFrame(summary_data).sort_values("MAP", ascending=False)

print("=" * 100)
print("TRAINING PERFORMANCE SUMMARY (50 queries)")
print("=" * 100)
print(summary_df.to_string(index=False))
print("=" * 100)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))

methods = summary_df["Method"].tolist()
maps = summary_df["MAP"].tolist()
colors = ["#3498db", "#2ecc71", "#e74c3c"]

bars = ax.bar(methods, maps, color=colors[:len(methods)], edgecolor="black")
ax.set_ylabel("MAP")
ax.set_title("Training Performance Comparison (50 queries)")
ax.set_ylim(0, max(maps) * 1.15)

for bar, m in zip(bars, maps):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.005,
            f"{m:.4f}", ha="center", va="bottom", fontsize=11)

plt.tight_layout()
plt.show()

## 7. Final Model Selection

In [ ]:
# Select best overall model
best_result = results_sorted[0]

print("Best model from all experiments:")
print(f"  Model: {best_result['model']}")
print(f"  MAP: {best_result['map']:.4f}")
print(f"  Config: {best_result['config_key']}")

# Find matching experiment configuration
best_model_name = best_result["model"]
best_config_key = best_result["config_key"]

matched_config = None
matched_model_class = None

for exp in EXPERIMENTS:
    if exp["model_name"] == best_model_name:
        matched_model_class = exp["model_class"]
        for params in exp["param_grid"]:
            config_key = generate_config_key(exp["model_name"], params)
            if config_key == best_config_key:
                matched_config = params.copy()
                break
        if matched_config:
            break

if matched_config is None:
    raise ValueError(f"Could not find matching config for {best_config_key}")

print(f"\n✓ Best model identified: {best_model_name}")
print(f"  Parameters: {matched_config}")

## 8. Generate Submission Files

### 8.1. Initialize Final Models

In [ ]:
# Get best parameters for each method
best_bm25_params = None
best_rm3_params = None
best_hybrid_params = None

for model_name, r in best_per_model.items():
    for exp in EXPERIMENTS:
        if exp["model_name"] == model_name:
            for params in exp["param_grid"]:
                if generate_config_key(model_name, params) == r["config_key"]:
                    if model_name == "BM25":
                        best_bm25_params = params
                    elif model_name == "RM3":
                        best_rm3_params = params
                    elif model_name == "HybridNeural":
                        best_hybrid_params = params
                    break

print("Final model configurations:")
print(f"  BM25: {best_bm25_params}")
print(f"  RM3: {best_rm3_params}")
print(f"  HybridNeural: {best_hybrid_params}")

### 8.2. Run Inference on Test Queries

In [ ]:
print(f"Running inference on {len(test_queries)} test queries...\n")

# Method 1: BM25
print("Method 1: BM25")
model_1 = BM25Retriever(**best_bm25_params)
run_1 = model_1.search(test_queries, k=1000)
print(f"  ✓ {len(run_1)} queries processed")

# Method 2: RM3
print("\nMethod 2: BM25 + RM3")
model_2 = RM3Retriever(**best_rm3_params)
run_2 = model_2.search(test_queries, k=1000)
print(f"  ✓ {len(run_2)} queries processed")

# Method 3: Hybrid Neural
print("\nMethod 3: Hybrid Neural Re-ranking")
model_3 = HybridNeuralRetriever(**best_hybrid_params)
run_3 = model_3.search(test_queries, k=1000)
print(f"  ✓ {len(run_3)} queries processed")

print("\n✓ All test queries processed")

### 8.3. Export TREC Format

In [ ]:
def write_trec_run(
    run: Dict[str, List[Tuple[str, float]]],
    filepath: str,
    run_name: str
):
    """
    Write run to TREC format.
    Format: topic_id Q0 doc_id rank score run_name
    """
    with open(filepath, "w") as f:
        for qid in sorted(run.keys(), key=lambda x: int(x)):
            results = run[qid]
            sorted_results = sorted(results, key=lambda x: x[1], reverse=True)
            for rank, (docid, score) in enumerate(sorted_results[:1000], start=1):
                f.write(f"{qid} Q0 {docid} {rank} {score:.6f} {run_name}\n")
    
    print(f"✓ Written: {filepath}")


print("Writing submission files...\n")

write_trec_run(run_1, "run_1.res", "run_1")
write_trec_run(run_2, "run_2.res", "run_2")
write_trec_run(run_3, "run_3.res", "run_3")

print("\n✓ All submission files written")

### 8.4. Validate Output

In [ ]:
def validate_run_file(filepath: str, expected_queries: int = 199, docs_per_query: int = 1000):
    """Validate TREC run file format and contents."""
    query_docs = defaultdict(list)
    
    with open(filepath, "r") as f:
        for line in f:
            parts = line.strip().split()
            qid, _, docid, rank, score, _ = parts
            query_docs[qid].append((int(rank), float(score)))
    
    assert len(query_docs) == expected_queries, f"Expected {expected_queries} queries, got {len(query_docs)}"
    
    for qid, docs in query_docs.items():
        assert len(docs) == docs_per_query, f"Query {qid}: expected {docs_per_query} docs, got {len(docs)}"
        scores = [s for _, s in sorted(docs, key=lambda x: x[0])]
        assert scores == sorted(scores, reverse=True), f"Query {qid}: scores not in decreasing order"
    
    print(f"✓ {filepath}: {len(query_docs)} queries × {docs_per_query} docs, scores non-increasing")


print("Validating submission files...\n")

validate_run_file("run_1.res")
validate_run_file("run_2.res")
validate_run_file("run_3.res")

print("\n" + "=" * 60)
print("SUBMISSION FILES READY")
print("=" * 60)
print("Files: run_1.res, run_2.res, run_3.res")
print("Format: TREC 6-column")
print("=" * 60)